In [1]:
import smart_wsi_scanner as sws

In [2]:
from smart_wsi_scanner.smartpath import init_pycromanager
core, studio = init_pycromanager()

In [10]:
if core.get_property("MicroPublisher6","Color")=="OFF":
    core.set_property("MicroPublisher6","Color","ON")
if core.get_property("MicroPublisher6","Color - Algorithm CFA") != "G-R-B-G":
    core.set_property("MicroPublisher6","Color - Algorithm CFA", "G-R-B-G")


In [ ]:
## setting one mode of operation : 0deg, 10x 

ConfigGroup,ColorCam,ColorOn 10x 0deg,MicroPublisher6,Color - Blue scale,1.0000
ConfigGroup,ColorCam,ColorOn 10x 0deg,MicroPublisher6,Color - Green scale,1.1691
ConfigGroup,ColorCam,ColorOn 10x 0deg,MicroPublisher6,Color - Red scale,1.8056
ConfigGroup,ColorCam,ColorOn 10x 0deg,MicroPublisher6,Exposure,500.0000
ConfigGroup,ColorCam,ColorOn 10x 0deg,DObjective,Label,10x 0.3

In [12]:
import platform
import psutil

if platform.system() != "Windows":
    print(platform)

for proc in psutil.process_iter(['name']):
    try:
        if "ImageJ.exe" in proc.info['name'] or "Micro-Manager" in proc.info['name']:
            print("runing")
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        pass


not running


In [21]:
for proc in psutil.process_iter(['name']):
    name = proc.info['name']
    if name.startswith('j'):print(name); break
    

javaw.exe


True